<img alt="Cover" src="./images/1.cover.png" width="2000" />

---

# Definition

### According to Wikipedia

<font size="5">
<ul>
<li>Parallel computing is a type of computing “in which many calculations or processes are carried out simultaneously”. </li>
<li>Concurrent computing is a form of computing in which several computations are executed concurrently – in overlapping time periods – instead of sequentially. </li>
<li>It is possible to have parallelism without concurrency, and concurrency without parallelism.</li>
</ul>
</font>

### Motivation

<font size="5">
<ul>
<li>Faster runtime</li>
<li>Improved responsiveness</li>
</ul>
</font>

---

# Parallelism vs Concurrency

<img alt="P vs C" src="./images/2.jpeg" width="2000" />

---

# Concurrency: processes vs threads

<img alt="P vs C" src="./images/3.jpeg" width="2000" />

---

# Preemptive vs cooperative scheduling

<img alt="P vs C" src="./images/4.jpeg" width="2000" />

---

# Parallel and Concurrent Programming in the JVM

<font size="5">
<ul>
    <li>The JVM has its own scheduler
        <ul>
        <li>It is independent from the OS scheduler</li>
        <li>A JVM thread != an OS thread</li>
        <li> => Multithreaded JVM apps can run on a single-threaded OS (DOS) </li>
        </ul>
    </li>
    <li>JVM threads are either daemons or user threads.</li>
    <li>The app stops when all user threads are done.</li>
    <li>The JVM does not wait for daemon threads to finish.</li>
</ul>
</font>

___

# Parallel programming in the JVM

## 2 Java packages

<font size="5">
<ul>
    <li>java.lang contains basic primitives: Runnable, Thread, etc.</li>
    <li>java.util.concurrent contains synchronization primitives and concurrent data structures.</li>
</ul>
</font>

## 1 Kotlin package

<font size="5">
<ul>
    <li>kotlin.concurrent — Wrappers and extensions for Java classes (quality-of-life).</li>
</ul>
</font>

___

# Throwback: Single Abstract Method interfaces

<font size="5">
SAM – an interface with a single abstract method, which can be instantiated with a lambda.

Java:
</font>

In [ ]:
@FunctionalInterface
public interface Runnable {
    public abstract void run();
}

<font size="5">Kotlin:</font>

In [ ]:
fun interface Screamer {
    fun scream(): Unit
}

<font size="5">Example:</font>

In [ ]:
class RunnableWrapper(val runnable: Runnable)
class ScreamerWrapper(val screamer: Screamer)

In [ ]:
val myWrapperFromLambda = RunnableWrapper { println("Runnable from lambda speaking!") }
val independentScreamer = Screamer { println("Boo!") }
val wrappedScreamer = ScreamerWrapper { println("Boo! x2") }

myWrapperFromLambda.runnable.run()
independentScreamer.scream()
wrappedScreamer.screamer.scream()

---

# Ways to create threads

<font size="5">
Java Standard Library provide the Thread class, which implements Runnable.
You can inherit from the Thread.
</font>

In [ ]:
class MyThread : Thread() {
    override fun run() {
        println("${currentThread()} is running")
    }
}

In [ ]:
val myThread = MyThread()
myThread.start()

# run vs start

<font size="5">

**Never call Thread.run()!**

run will execute the body _on your thread_, while start will create _a new thread_ where run's body will be executed.
</font>

In [ ]:
println("${Thread.currentThread()} is the main thread")

val myThread1 = MyThread()
val myThread2 = MyThread()
val myThread3 = MyThread()

myThread1.start()
myThread2.run()
myThread3.start()

<font size="5">If the code above gets executed, most probably you see the main thread twice and then two other threads. This means that myThread2 run's body was executed on the main thread. 
Two other threads took some time to start and thus appear later.</font>

<font size="5">You can implement the Runnable interface and pass it to a thread. You can pass the same Runnable to several threads. </font>




In [ ]:
var extremelyBadGlobalVariable = 0
val myRunnable = Runnable { println("Runnable ${++extremelyBadGlobalVariable}") }

In [ ]:
val thread1 = Thread(myRunnable)
val thread2 = Thread(myRunnable)

thread1.start()
thread2.start()

<font size="5">In Kotlin the preferrable way to create threads is with a simple HOF:</font>

In [ ]:
import kotlin.concurrent.thread

val kotlinThread = thread {
    println("I start instantly, but you can pass an option to start me manually")
}

val kotlinThreadLater = thread(start = false) {
    println("I do not start instantly, you have to start me manually")
}

In [ ]:
kotlinThreadLater.start()

---

# Thread properties

<font size="5">A thread's properties cannot be changed after it is started. </font>

<font size="5">Main properties of a thread:</font>

In [ ]:
kotlinThread.id // Long, This is the thread's identifier
kotlinThread.name // String
kotlinThread.priority // Int, This can range from 1 to 10, with a larger value indicating higher priority
kotlinThread.isDaemon // Boolean
kotlinThread.state // Thread.State
kotlinThread.isAlive // Boolean

# State of a thread

| state         | isAlive |
|---------------|---------|
| NEW           | false   |
| RUNNABLE      | true    |
| BLOCKED       | true    |
| WAITING       | true    |
| TIMED_WAITING | true    |
| TERMINATED    | false   |


# 

<img alt="Thread states" src="./images/5.png" width="2000" />

___

# Ways to manipulate a thread's state

In [26]:
val simple = thread { /* code */ } // Creates a new thread

val myThread = thread(false) { /* code */ } // Creates a new thread in the NEW state
myThread.start() // Starts the thread
myThread.join() // Causes the current thread to wait for myThread to finish
val durationInMs: Long = 3000
Thread.sleep(durationInMs) // Puts the current thread to sleep for 3 seconds
Thread.yield() // Tries (sic!) to step back
myThread.interrupt() // Tries (sic!) to interrupt a thread
myThread.isInterrupted() // Boolean, Checks whether myThread was interrupted
Thread.interrupted() // Boolean, Checks and clears the interruption flag

false

 - <font size="5">The sleep and yield methods are only applicable to the current thread, which means that you cannot suspend another thread.</font>
 - <font size="5">All blocking and waiting methods can throw InterruptedException.</font>

In [28]:
import java.lang.Thread.sleep

In [32]:
val runningThread = thread {
    var counter = 0
    try {
        while (true) {
            sleep(100)
            println("Had some sleep $counter")
            counter++
        }
    } catch (e: Exception) {
        println(e)
    }
}
sleep(400)
runningThread.interrupt()

Had some sleep 0
Had some sleep 1
Had some sleep 2
java.lang.InterruptedException: sleep interrupted
sleep interrupted


# Classic worker

In [33]:
class ClassicWorker : Runnable {
    override fun run() {
        try {
            while (!Thread.interrupted()) {
                // do stuff
            }
        } catch (e: InterruptedException) {} // absolutely legal empty catch block
    }
}

<font size="5">Worker runs indefinitely. In the body it should get work from somewhere, execute it, and then go on to loop. In case it is interrupted it finishes more or less gracefully.</font>

___

# Parallelism and shared memory

## Examples of problematic interleaving

<font size="5">Parallel threads have access to the same shared memory.</font>

<font size="5">This often leads to problems that cannot arise in a single-threaded environment.</font>

In [34]:
class Counter {
    private var c = 0

    fun increment() {
        c++
    }
    
    fun decrement() {
        c--
    }
    
    fun value(): Int {
        return c
    }
}

<font size="5">Both operations on c are single, simple statements.</font>

<font size="5">However, even simple statements can be translated into multiple steps by the virtual machine, and those steps can be interleaved.</font>


In [42]:
val sharedCounter = Counter()

val thread1 = thread {
    sharedCounter.increment()
}
val thread2 = thread {
    sharedCounter.increment()
}

thread1.join()
thread2.join()
println(sharedCounter.value())

2


<font size="5">Suppose both thread1 and thread2 invoke increment at the same time. If the initial value of c is 0, their interleaved actions might follow this sequence:</font>
- <font size="5"> T#1: Read value 0 from c.</font>
- <font size="5"> T#2: Read value 0 from c.</font>
- <font size="5"> T#1: Increment value — result is 1.</font>
- <font size="5"> T#1: Write result 1 to c.</font>
- <font size="5"> T#2: Increment value — result is 1.</font>
- <font size="5"> T#2: Write result 1 to c.</font>

---

# Synchronization mechanisms
- <font size="5"> Mutual exclusion, such as Lock and the synchronized keyword.</font>
- <font size="5"> Concurrent data structures and synchronization primitives.</font>
- <font size="5"> Atomics, which work directly with shared memory (DANGER ZONE).</font>

# Locks

In [43]:
import java.util.concurrent.locks.ReentrantLock
import kotlin.concurrent.withLock

val exampleLock = ReentrantLock()

exampleLock.lock() // Acquires the lock
exampleLock.tryLock() // Boolean, Tries to acquire the lock
exampleLock.unlock() // Releases the lock
exampleLock.withLock {
    // critical section
} // Executes a lambda with the lock held (has try/catch inside)
val exampleCondition = exampleLock.newCondition() // Creates a condition variable associated with the lock

In [44]:
class LockedCounter {
    private var c = 0

    private val lock = ReentrantLock()

    fun increment() {
        lock.withLock { c++ }
    }

    fun decrement() {
        lock.withLock { c-- }
    }

    fun value(): Int {
        lock.withLock { return c }
    }
}

# Conditions

<font size="5">A condition allows a thread holding a lock to wait until another thread signals it about a certain event. Internally, the await method releases the associated lock upon call, and acquires it back before finally returning it again.</font>

In [45]:
class PositiveLockedCounter {
    private var c = 0
    private val lock = ReentrantLock()
    private val condition = lock.newCondition()

    fun increment() {
        lock.withLock {
            c++
            condition.signal()
        }
    }

    fun decrement() {
        lock.withLock {
            while (c == 0) {
                condition.await()
            }
            c--
        }
    }

    fun value(): Int {
        return lock.withLock { c }
    }
}

In [49]:
val posCounter = PositiveLockedCounter()

val decrementor = { 
    println("Trying to decrement the counter...")
    posCounter.decrement()
    println("Successfully decremented!")
}
val incrementor = {
    println("Going to sleep and the increment the counter")
    sleep(100)
    posCounter.increment()
    println("Incremented.")
}

val threads = buildList {
    repeat(5) {
        add(thread(block = decrementor))
        add(thread(block = incrementor))
    }
}

threads.forEach { it.join() }

Trying to decrement the counter...
Going to sleep and the increment the counter
Trying to decrement the counter...
Trying to decrement the counter...
Going to sleep and the increment the counter
Going to sleep and the increment the counter
Trying to decrement the counter...
Going to sleep and the increment the counter
Trying to decrement the counter...
Going to sleep and the increment the counter
Incremented.
Successfully decremented!
Incremented.
Successfully decremented!
Incremented.
Successfully decremented!
Incremented.
Successfully decremented!
Incremented.
Successfully decremented!


# ReentrantLock

<font size="5">Allows the lock to be acquired multiple times by the same thread. </font>

In [57]:
class ReentrantExample {
    private val lock = ReentrantLock()

    fun criticalSection() = lock.withLock {
        lock.withLock {
            println("${Thread.currentThread()} holds ${lock.holdCount}")
            sleep(1000)
        }
    }

    fun anotherFunction() = lock.withLock {
        criticalSection()
    }

    val isFair: Boolean
        get() = lock.isFair // Checks the fairness of the lock, constructor of ReentrantLock accepts `fair: Boolean` argument, false by default

    val queueLength: Int
        get() = lock.queueLength
}

In [58]:
val reentrantExample = ReentrantExample()

val t1 = thread {
    reentrantExample.anotherFunction()
}
val t2 = thread {
    reentrantExample.criticalSection()
}
val t3 = thread {
    reentrantExample.criticalSection()
}
sleep(100)
println("Q length: ${reentrantExample.queueLength}")
println("Lock is fair: ${reentrantExample.isFair}")
t1.join()
t2.join()
t3.join()

Thread[Thread-333,5,main] holds 3
Q length: 2
Lock is fair: false
Thread[Thread-334,5,main] holds 2
Thread[Thread-335,5,main] holds 2


# ReadWriteLock

<font size="5">ReadWriteLock allows multiple readers to access a resource concurrently but only lets a single writer modify it.</font>

In [61]:
import java.util.concurrent.locks.*
import kotlin.concurrent.read
import kotlin.concurrent.write

val rwLock: ReadWriteLock = ReentrantReadWriteLock()

val readLock: Lock = rwLock.readLock()
val writeLock: Lock = rwLock.writeLock()

In [62]:
class PositiveLockedCounter {
    private var c = 0
    private val rwLock = ReentrantReadWriteLock()

    fun increment() {
        rwLock.write { c++ }
    }

    fun decrement() {
        rwLock.write { c-- }
    }

    fun value(): Int {
        return rwLock.read { c }
    }
}


---

# The synchronized statemend

<font size="5">In the JVM, every object has an intrinsic lock associated with it (aka a monitor).</size>

In [ ]:
class SynchronizedCounter {
    private var c = 0

    fun increment() {
        synchronized(this) { c++ }
    }
}

# synchronized method

<font size="5">Java:</font>

In [ ]:
public class SynchronizedJavaCounter {
    private int c = 0;

    public synchronized void increment() {
        c++;
    }
}


<font size="5">Kotlin:</font>

In [ ]:
class SynchronizedKotlinCounter {
    private var c = 0

    @Synchronized
    fun increment() {
        c++
    }
}


---

# Concurrent blocking collections

<font size="5">java.util.concurrent is a Java package that implements both blocking and non-blocking concurrent collections, such as:</font>
- <font size="5">SynchronousQueue – One-element rendezvous channel </font>
- <font size="5">ArrayBlockingQueue – Fixed-capacity queue </font>
- <font size="5">LinkedBlockingQueue – Unbounded blocking queue </font>
- <font size="5">PriorityBlockingQueue – Unbounded blocking priority queue
</font>

# Concurrent non-blocking collections

<font size="5">java.util.concurrent is a Java package that implements both blocking and non-blocking concurrent collections, such as:</font>
- <font size="5">ConcurrentLinkedQueue – Non-blocking unbounded queue </font>
- <font size="5">ConcurrentLinkedDequeue – Non-blocking unbounded dequeue </font>
- <font size="5">ConcurrentHashMap – Concurrent unordered hash-map </font>
- <font size="5">ConcurrentSkipListMap – Concurrent sorted hash-map</font>

# Synchronization primitives

<font size="5">java.util.concurrent also implements concurrent data structures and synchronization primitives, such as:</font>
- <font size="5">Exchanger – Blocking exchange  </font>
- <font size="5">Phaser – Barrier synchronization</font>

---

# Java Memory Model: Weak behaviors

<font size="5">There are no guarantees when it comes to ordering!</font>

In [63]:
var x = 0
var y = 0


thread {
    x = 1
    y = 1
}

thread {
    val a = y
    val b = x

    println("$a, $b")
}

<font size="5">Possible outputs:</font>
- <font size="5"> 0, 0</font>
- <font size="5"> 0, 1</font>
- <font size="5"> 1, 1</font>
- <font size="5"> 1, 0 (???)</font>

<font size="5">There are no guarantees when it comes to progress!</font>

In [70]:
var flag = false

val t1 = thread {
    while (!flag) { }
    println("I am free!")
}

val t2 = thread { 
    flag = true
}

t1.join()
t2.join()

I am free!


<font size="5">Possible outputs:</font>
- <font size="5">"I am free!"</font>
- <font size="5">Halt</font>

---

Because the compiler is allowed to do this:

In [ ]:
thread {
    while (true) {}
    println("I am free!")
}

---

# JMM: Data-Race-Freedom Guarantee

<font size="5">But what does JMM guarantee?</font>

<font size="5">**Well-synchronized** programs have **simple interleaving semantics**.</font>

<font size="5">Well-synchronized = Data-race-free </font>

<font size="5">Simple interleaving semantics = Sequentially consistent semantics </font>

<font size="5">**Data-race-free** programs have **sequentially consistent semantics**. </font>

---

# Volatile fields

<font size="5">Volatile fields can be used to restore **sequential consistency**.</font>

In [ ]:
class OrderingTest {
    @Volatile var x = 0
    @Volatile var y = 0
    fun test() {
        thread {
            x = 1
            y = 1
        }
        thread {
            val a = y
            val b = x
            println("$a, $b")
        }
    }
}

In [ ]:
class ProgressTest {
    @Volatile var flag = false
    fun test() {
        thread {
            while (!flag) {}
            println("I am free!")
        }
        thread { flag = true }
    }
}

<font size="5">Volatile variables can be used for synchronization.</font>

In [ ]:
class OrderingTest {
    var x = 0
    @Volatile var y = 0
    fun test() {
        thread {
            x = 1
            y = 1
        }
        thread {
            val a = y
            val b = x
            println("$a, $b")
        }
    }
}

<font size="5">How do we know there is enough synchronization?</font>

---

# JMM: Happens-before relation

<img alt="Thread states" src="./images/6.png" width="720" />

---

# 

<img alt="Thread states" src="./images/7.png" width="720" />

---

#

<img alt="Thread states" src="./images/8.png" width="720" />

---

# JMM: Synchronizing actions

- <font size="5">Read and write for volatile fields </font>
- <font size="5">Lock and unlock</font>
- <font size="5">Thread run and start, as well as finish and join</font>

In [75]:
class Storage {
    @Volatile
    var x = 23
    private var y = 42
    val lock = ReentrantReadWriteLock()
    
    fun getStored() = lock.read { y }
    
    fun store(newY: Int) = lock.write { y = newY }
    
    fun print(marker: String? = null) = lock.read { println("${marker?.let { "$it: " } ?: ""}$x, $y") }
}

In [81]:
val s = Storage()
s.print() // 23, 42
val t1 = thread {
    s.store(44)
    s.x = -1
    s.print("t1")
}
val t2 = thread {
    s.print("t2")
}
val t3 = thread {
    s.x = 777
    s.store(66)
}

t1.join()
t2.join()
s.print() // never 23, 42
t3.join()
s.print()

23, 42
t1: -1, 44
t2: -1, 44
-1, 44
777, 66


---

# JMM: DRF-SC again

<font size="5">Two events form a data race if: </font>
- <font size="5">Both are memory accesses to the same field. </font>
- <font size="5">Both are **plain** (non-atomic) accesses.</font>
- <font size="5">At least one of them is a *write* event.</font>
- <font size="5">They are not related by *happens before*.</font>


<font size="5">Data-race-free programs have sequentially consistent semantics.</font>

<font size="5">A program is data-race-free if, **for every possible execution** of this program, **no** two events form a **data race**.</font>

---

# JMM: Atomics aka Danger Zone

In [82]:
import java.util.concurrent.atomic.AtomicInteger

class AtomicCounter {
    private val c = AtomicInteger()

    fun increment() {
        c.incrementAndGet()
    }

    fun decrement() {
        c.decrementAndGet()
    }

    fun value(): Int {
        return c.get()
    }
}

# TODO